# Setup

In [ ]:
import re
import pprint
import time
import pandas as pd
import numpy as np
from sklearn.model_selection import ParameterGrid
import torch

from transformers import (
    RobertaModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    get_linear_schedule_with_warmup,
    set_seed,
    BertModel,
    BertTokenizer,
    AdamW
)

from peft import (
    PeftModel,
    PeftConfig,
    get_peft_config,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    LoraConfig,
    PeftType,
    PrefixTuningConfig,
    PromptEncoderConfig,
)

from functions import *
from data_scripts import get_setups


log_file_path = os.path.join('', 'execution_time_log.txt')
log_file = open(log_file_path, 'a')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

gpu_id = 0
total_memory = torch.cuda.get_device_properties(gpu_id).total_memory
used_memory = torch.cuda.memory_allocated(gpu_id)
reserved_memory = torch.cuda.memory_reserved(gpu_id)

# Impresión de información sobre la memoria de la GPU
print(f"Total memory: {total_memory / 1e9} GB")
print(f"Used memory: {used_memory / 1e9} GB")
print(f"Reserved memory: {reserved_memory / 1e9} GB")

EPOCHS = 1

# Registrar el tiempo de carga de datos
data_load_start_time = time.time()
experiments = get_setups()
data_load_end_time = time.time()
log_file.write(f"Data loading time: {data_load_end_time - data_load_start_time} seconds\n")

params_grid = ParameterGrid({
    'setup': list(experiments.items()),
    'structure': ['peft'],
    'seed': [97,199, 103, 23, 137],
    'regularizer': [1, 0],
    'divergence_metric': ['kl', 'tv'],
    'distribution': ['Uniform', 'Beta'],
})

peft_config = LoraConfig(
    task_type="SEQ_CLS",
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1
)
base_path = 'baseline'
excel_file = base_path+'/eval.xlsx'

# Train

In [ ]:
# Ejecución del experimento para cada configuración de parámetros
for params in params_grid:

    print('params:\n')
    pprint.pprint(params)

    all_results = []
    setup = params['setup'][1]
    N = 0
    sheet_name = f'seed {params["seed"]}'

    # Obtener k_classes del dataset de entrenamiento
    generalisation = []
    train_dataloader = setup['train_dataloader']
    labels = []
    for batch in train_dataloader:
        N += batch['input_ids'].shape[0]
        generalisation.extend(batch['generalisation'].numpy())
        labels.extend(batch['labels'].numpy())

    mask_id = np.array(generalisation) == 1
    k_classes = len(set(np.array(labels)[mask_id]))

    # Configuración del dispositivo y la pérdida personalizada
    device = prepare_environment(seed=params['seed'], device="cuda")
    custom_loss_function = CustomLoss(
        loss_type=params['divergence_metric'],
        weight_regularizer=params['regularizer'],
        num_classes=k_classes,
        distribution=params['distribution']
    )

    model, optimizer = initialize_model(params['structure'], k_classes, peft_config, device)

    # Entrenamiento y evaluación del modelo
    flag_peft = True if params['structure'] == 'peft' else False
    start_time_run = time.time()
    val_metrics, train_metrics = train_evaluate(
        model,
        setup['train_dataloader'],
        setup['eval_dataloader'],
        optimizer,
        EPOCHS,
        flag_peft,
        custom_loss_function
    )
    end_time_run = time.time()
    log_file.write(
        f"Training and validation time for Normal, params:\n {params}.\n {start_time_run - end_time_run} seconds\n"+"--"*20
    )
    print(f"Training and validation time for Normal, params:\n {params}.\n {start_time_run - end_time_run} seconds\n"+"--"*20)

    total_time = end_time_run - start_time_run
    minutes = int(total_time // 60)
    seconds = int(total_time % 60)

    print(f'Total time: {minutes}:{seconds}')
    print(train_metrics)
    print(val_metrics)

    model_path_saved = save_results(model, val_metrics, train_metrics, params, base_path, k_classes)
    params_str = ', '.join([f'({key}:{value})' for key, value in params.items()])

    # Guardar resultados adicionales según el valor de regularizer
    if params['regularizer'] == 0:
        # MSP
        metrics = eval_model(model, setup['eval_dataloader'])
        metrics['tech'] = 'MSP'
        metrics['params'] = params_str
        all_results.append(metrics)

        # ViM
        for D in [k_classes, 256, 512]:
            u, NS, alpha = get_vim_model(model, setup['train_dataloader'], dim=D)
            metrics = test_model_vim(model, setup['eval_dataloader'], alpha, u, NS)
            metrics['tech'] = f'ViM(dim={D})'
            metrics['params'] = params_str
            all_results.append(metrics)
    else:
        # OE + Uniform
        metrics = eval_model(model, setup['eval_dataloader'])
        metrics['tech'] = 'OE+'+params['distribution']
        metrics['params'] = params_str
        all_results.append(metrics)

    # Guardar resultados en Excel
    if not os.path.exists(excel_file):
        new_df = pd.DataFrame(columns=list(metrics.keys()))
        for dic_data in all_results:
            new_df.loc[len(new_df)] = list(dic_data.values())
        new_df.to_excel(excel_file, sheet_name=sheet_name)
    else:
        with pd.ExcelWriter(excel_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            if sheet_name in writer.sheets:
                df = pd.read_excel(excel_file, sheet_name=sheet_name)
            else:
                df = pd.DataFrame(columns=list(metrics.keys()))
            # Asegurarse de que las columnas de df coincidan con las claves del diccionario
            for dic_data in all_results:
                if not set(dic_data.keys()).issubset(df.columns):
                    raise ValueError(
                        f"Las claves del diccionario no coinciden con las columnas del DataFrame.\nClaves del diccionario: {dic_data.keys()}\nColumnas del DataFrame: {df.columns}")

                # Asegúrate de que las columnas estén en el orden correcto
                df = df.reindex(columns=dic_data.keys())

                # Añadir la fila de datos al DataFrame
                df.loc[len(df)] = list(dic_data.values())
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    torch.cuda.empty_cache()

# Testing

In [ ]:
import os

# Configuración de PEFT
peft_config = LoraConfig(
    task_type="SEQ_CLS",
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.1
)

# Ruta al archivo de prueba
test_excel_file = base_path + '/test_propose_v1.xlsx'

# Fase de Testing Adicional
for name_setup, dataloaders in list(experiments.items())[3:]:
    train_dataloader = dataloaders['train_dataloader']
    evaluation_dataloader = dataloaders['test_dataloader']

    path_to_models = f'{base_path}/models/'
    model_files = os.listdir(path_to_models)
    all_results = []
    sheet_name = name_setup

    # Filtrar los archivos de modelos que contienen "20NG" en su nombre
    filtered_models = [model for model in model_files if ("NG20" in model) and ("distribution:Beta" in model)]

    for selected_model_path in filtered_models:
        print(selected_model_path)
        if name_setup not in selected_model_path:
            continue

        selected_model_path = os.path.join(path_to_models, selected_model_path)
        print(f"Procesando el modelo: {selected_model_path}")

        match_seed = re.search(r'seed:(\d+)', selected_model_path)
        k_classes = re.search(r'k: (\d+)', selected_model_path)
        distribution_match = re.search(r'distribution:([a-zA-Z]+)', selected_model_path)
        distribution_name = distribution_match.group(1)
        seed_model = int(match_seed.group(1))

        params_str = re.findall(r'\((.*?)\).pth', selected_model_path)[0]
        params_dict = {}

        pattern = r'(\w+):(\([^)]+\)(?:,\s*\([^)]+\))*|[^\,]+)'

        matches = re.finditer(pattern, params_str)
        for match in matches:
            key = match.group(1).strip()
            value = match.group(2).strip()
            try:
                value = eval(value)
            except:
                pass
            params_dict[key] = value

        model = AutoModelForSequenceClassification.from_pretrained(
            'roberta-base',
            num_labels=int(k_classes.group(1)),
            return_dict=True
        )
        model = get_peft_model(model, peft_config)
        model.load_state_dict(torch.load(selected_model_path))
        model.to(device)

        if 'regularizer:0' in selected_model_path:
            # MSP
            metrics = eval_model(model, evaluation_dataloader)
            metrics['tech'] = 'MSP'
            metrics.update(params_dict)
            all_results.append(metrics)

            # ViM
            D = 256
            u, NS, alpha = get_vim_model(model, train_dataloader, dim=D)
            metrics = test_model_vim(model, evaluation_dataloader, alpha, u, NS)
            metrics['tech'] = f'ViM(dim={D})'
            metrics.update(params_dict)
            all_results.append(metrics)
        else:
            # OE + Uniform
            metrics = eval_model(model, evaluation_dataloader)
            metrics['tech'] = 'OE+Beta'
            metrics.update(params_dict)
            all_results.append(metrics)

    if len(all_results) == 0:
        continue

    # Guardar resultados de Testing en Excel
    if not os.path.exists(test_excel_file):
        new_df = pd.DataFrame(columns=list(metrics.keys()))
        for dic_data in all_results:
            new_df.loc[len(new_df)] = list(dic_data.values())
        new_df.to_excel(test_excel_file, sheet_name=sheet_name, index=False)
    else:
        with pd.ExcelWriter(test_excel_file, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
            # Verificar si la hoja ya existe
            if sheet_name in writer.sheets:
                df = pd.read_excel(test_excel_file, sheet_name=sheet_name)
            else:
                df = pd.DataFrame(columns=list(metrics.keys()))
    
            # Eliminar la columna 'Unnamed: 0' si existe
            if 'Unnamed: 0' in df.columns:
                df.drop(columns=['Unnamed: 0'], inplace=True)
    
            for dic_data in all_results:
                # Verificar si las claves del diccionario coinciden con las columnas del DataFrame
                missing_cols = set(dic_data.keys()) - set(df.columns)
                if missing_cols:
                    # Agregar columnas faltantes
                    for col in missing_cols:
                        df[col] = None  # Rellenar con valores por defecto (None o np.nan)
    
                # Asegúrate de que las columnas estén en el orden correcto
                ordered_values = [dic_data.get(col, None) for col in df.columns]
    
                # Añadir la fila de datos al DataFrame
                df.loc[len(df)] = ordered_values
    
            # Guardar el DataFrame actualizado en la hoja correspondiente
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    torch.cuda.empty_cache()

In [ ]:
aux = pd.read_excel(base_path + '/test_propose_v1.xlsx')
aux

In [ ]:
# Lista de columnas a considerar
columns_to_consider = [
    'acc', 'f1_m', 'f1_M', 'AUROC_ID/far', 'FPR95_ID/far',
    'AUROC_ID/near', 'FPR95_ID/near', 'AUROC_near/far',
    'FPR95_near/far', 'AUROC_ID/near+far', 'FPR95_ID/near+far'
]

# Filtrar solo las columnas deseadas
filtered_aux = aux[columns_to_consider]

# Calcular la media de cada columna
mean_values = filtered_aux.mean()

# Calcular la desviación estándar de cada columna
std_values = filtered_aux.std()

# Combinar en un DataFrame con nombres de métricas
summary = pd.DataFrame({
    'Metric': mean_values.index,
    'Mean': mean_values.values,
    'Standard Deviation': std_values.values
})

# Mostrar el resumen
print(summary)